# Swan Consulting

In [4]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
# read data
df = pd.read_csv('Swan_Project_Data.csv')

In [14]:
pd.set_option('display.max_columns', None)

In [16]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,Moved
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,Competitor had better devices


In [20]:
df['Churn Value'].unique()

array([1, 0], dtype=int64)

In [12]:
df.dtypes

CustomerID            object
Count                  int64
Country               object
State                 object
City                  object
Zip Code               int64
Lat Long              object
Latitude             float64
Longitude            float64
Gender                object
Senior Citizen        object
Partner               object
Dependents            object
Tenure Months          int64
Phone Service         object
Multiple Lines        object
Internet Service      object
Online Security       object
Online Backup         object
Device Protection     object
Tech Support          object
Streaming TV          object
Streaming Movies      object
Contract              object
Paperless Billing     object
Payment Method        object
Monthly Charges      float64
Total Charges         object
Churn Label           object
Churn Value            int64
Churn Reason          object
dtype: object

In [22]:
import re


In [54]:
[str(i).replace(" ", "") for i in df["Total Charges"]]

['108.15',
 '151.65',
 '820.5',
 '3046.05',
 '5036.3',
 '528.35',
 '39.65',
 '20.15',
 '4749.15',
 '30.2',
 '1093.1',
 '316.9',
 '3549.25',
 '1105.4',
 '144.15',
 '1426.4',
 '633.3',
 '1752.55',
 '857.25',
 '79.35',
 '496.9',
 '927.35',
 '113.85',
 '2514.5',
 '7611.85',
 '5238.9',
 '2868.15',
 '1064.65',
 '229.55',
 '2135.5',
 '1502.65',
 '35.45',
 '81.25',
 '70.45',
 '563.65',
 '45.65',
 '655.5',
 '1592.35',
 '273',
 '2651.2',
 '242.8',
 '5000.2',
 '1145.7',
 '503.6',
 '29.95',
 '662.65',
 '1111.65',
 '24.8',
 '531',
 '79.9',
 '1934.45',
 '84.5',
 '268.35',
 '179.35',
 '55.2',
 '84.6',
 '114.1',
 '1120.3',
 '1074.3',
 '48.55',
 '147.15',
 '40.2',
 '80.55',
 '132.2',
 '1364.3',
 '1099.6',
 '1511.2',
 '24.8',
 '70.6',
 '401.1',
 '5624.85',
 '1779.95',
 '70',
 '606.55',
 '435.4',
 '1712.7',
 '73.6',
 '396.1',
 '2303.35',
 '415.55',
 '3058.65',
 '2231.05',
 '4350.1',
 '139.05',
 '511.25',
 '155.8',
 '5293.95',
 '4759.85',
 '6603',
 '168.2',
 '371.9',
 '840.1',
 '846',
 '805.1',
 '492.55',

In [64]:
[float(x) for x in df['Total Charges']]

ValueError: could not convert string to float: ' '

In [56]:
empty_string_indices = df[df['Total Charges'] == ' '].index

In [66]:
df.columns

Index(['CustomerID', 'Count', 'Country', 'State', 'City', 'Zip Code',
       'Lat Long', 'Latitude', 'Longitude', 'Gender', 'Senior Citizen',
       'Partner', 'Dependents', 'Tenure Months', 'Phone Service',
       'Multiple Lines', 'Internet Service', 'Online Security',
       'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
       'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method',
       'Monthly Charges', 'Total Charges', 'Churn Label', 'Churn Value',
       'Churn Reason'],
      dtype='object')

In [70]:
df.head(2)

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved


In [77]:
yn_cols = ['Senior Citizen','Partner', 'Dependents', 'Phone Service','Multiple Lines','Online Security',
           'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV','Streaming Movies', 
           'Paperless Billing']
for col in yn_cols:
    col_name = col.replace(' ', '_')
    df[f'fe_{col_name}'] = df[f'{col}'].map({'No':0, 'Yes':1, 0:0, 1:1})

In [79]:
df.head()

,CustomerID,Count,Country,State,City,Zip Code,Lat Long,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,Device Protection,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Reason,fe_Senior_Citizen,fe_Partner,fe_Dependents,fe_Phone_Service,fe_Multiple_Lines,fe_Online_Security,fe_Online_Backup,fe_Device_Protection,fe_Tech_Support,fe_Streaming_TV,fe_Streaming_Movies,fe_Paperless_Billing
0,3668-QPYBK,1,United States,California,Los Angeles,90003,"33.964131, -118.272783",33.964131,-118.272783,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,1,Competitor made better offer,0,0,0,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1
1,9237-HQITU,1,United States,California,Los Angeles,90005,"34.059281, -118.30742",34.059281,-118.307420,Female,No,No,Yes,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,1,Moved,0,0,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,9305-CDSKC,1,United States,California,Los Angeles,90006,"34.048013, -118.293953",34.048013,-118.293953,Female,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,1,Moved,0,0,1,1,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1
3,7892-POOKP,1,United States,California,Los Angeles,90010,"34.062125, -118.315709",34.062125,-118.315709,Female,No,Yes,Yes,28,Yes,Yes,Fiber optic,No,No,Yes,Yes,Yes,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,1,Moved,0,1,1,1,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1
4,0280-XJGEX,1,United States,California,Los Angeles,90015,"34.039224, -118.266293",34.039224,-118.266293,Male,No,No,Yes,49,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Bank transfer (automatic),103.70,5036.3,Yes,1,Competitor had better devices,0,0,1,1,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1
